In [1]:
from tensorflow.keras.layers import Input, SimpleRNN, Dense, Flatten,TimeDistributed,LSTM,Masking,Bidirectional
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import SGD, Adam

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline
import csv
from sklearn.model_selection import train_test_split
import scispacy
import spacy
nlp = spacy.load("en_core_sci_md", disable = ['parser','tagger','ner'])

In [2]:
n = 2
tuple_sent = []
with open('C:/Users/Helen/python_crap/an_border101220.tsv', newline='')as f:
    reader = csv.reader(f, delimiter = '\t')
    for line in reader:
        senten = ([tuple(line[i:i + n]) for i in range(0, len(line), n)])
        senten = [(item[0], int(item[1])) for item in senten]
        tuple_sent.append(senten)


In [3]:
X_set = []
y_set = []
X_item = []
y_item = []
for sentences in tuple_sent:
    for tups in sentences:
        X_item.append(tups[0])
        y_item.append(tups[1])
    X_set.append(X_item)
    y_set.append(y_item)
    X_item = []
    y_item = []
    

In [4]:
def find_max_list(list):
    list_len = [len(i) for i in list]
    return(max(list_len))




In [5]:
def word_to_vector(X_set):
    n_item = len(X_set)
    #max_len = find_max_list(X_set)
    max_len = 136
    seq_dim = 200
    w_vecs = np.zeros((n_item,max_len,seq_dim))
    for i in range(0, len(X_set)):
        for j in range(0,len(X_set[i])):
            w_vecs[i,j] = nlp(X_set[i][j]).vector
    print(w_vecs.shape)
    return(w_vecs)



In [6]:
def y_to_number(y_set):
    y_mset = []
    y_item = []
    for i in range(len(y_set)):
        for j in range(len(y_set[i])):
            if y_set[i][j]==0:
                item = 1
                y_item.append(item)
            elif y_set[i][j]==1:
                item = 2
                y_item.append(item)
            else:
                item = 3
                y_item.append(item)
        y_mset.append(y_item)
        y_item = []
    y1 = np.array(y_mset)
    y1 = tf.keras.preprocessing.sequence.pad_sequences(y1, padding="post")
    y1 = tf.keras.utils.to_categorical(y1, num_classes=4, dtype='float32')
    print(y1.shape)
    return(y1)


In [8]:
y1 = np.array(y_mset)
y1 = tf.keras.preprocessing.sequence.pad_sequences(y1, padding="post")
y1 = tf.keras.utils.to_categorical(y1, num_classes=4, dtype='float32')
print(y1.shape)
##################
'''w_vecs = np.zeros((len(X_set),max_len,200))
for i in range(0,len(X_set)):
    for j in range(0,len(X_set[i])):
        w_vecs[i,j] = nlp(X_set[i][j]).vector
    print(w_vecs.shape)'''


(404, 136, 4)


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_set, y_set, test_size=0.33)
X_tr_pr = word_to_vector(X_train)
y_tr_pr = y_to_number(y_train)

(270, 136, 200)
(270, 136, 4)


In [8]:
###### create LSTM
model = Sequential()
model.add(Masking(mask_value = 0.,input_shape=(136, 200)))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(TimeDistributed(Dense(4, activation = 'softmax')))
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr = 0.01), metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, 136, 200)          0         
_________________________________________________________________
bidirectional (Bidirectional (None, 136, 256)          336896    
_________________________________________________________________
time_distributed (TimeDistri (None, 136, 4)            1028      
Total params: 337,924
Trainable params: 337,924
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
model.fit(X_tr_pr, y_tr_pr,epochs= 100)

Epoch 1/100
9/9 [==============================] - 2s 263ms/step - loss: 0.1685 - accuracy: 0.5946
Epoch 2/100
9/9 [==============================] - 2s 251ms/step - loss: 0.1045 - accuracy: 0.7638
Epoch 3/100
9/9 [==============================] - 2s 242ms/step - loss: 0.0803 - accuracy: 0.8510
Epoch 4/100
9/9 [==============================] - 2s 242ms/step - loss: 0.0640 - accuracy: 0.8728
Epoch 5/100
9/9 [==============================] - 2s 236ms/step - loss: 0.0504 - accuracy: 0.9023
Epoch 6/100
9/9 [==============================] - 2s 242ms/step - loss: 0.0401 - accuracy: 0.9242
Epoch 7/100
9/9 [==============================] - 2s 247ms/step - loss: 0.0334 - accuracy: 0.9375
Epoch 8/100
9/9 [==============================] - 2s 240ms/step - loss: 0.0274 - accuracy: 0.9503
Epoch 9/100
9/9 [==============================] - 2s 242ms/step - loss: 0.0253 - accuracy: 0.9554
Epoch 10/100
9/9 [==============================] - 2s 244ms/step - loss: 0.0196 - accuracy: 0.9678
Epoch 11/

9/9 [==============================] - 2s 256ms/step - loss: 2.0642e-04 - accuracy: 0.9995
Epoch 81/100
9/9 [==============================] - 2s 267ms/step - loss: 2.0853e-04 - accuracy: 0.9992
Epoch 82/100
9/9 [==============================] - 3s 282ms/step - loss: 1.7772e-04 - accuracy: 0.9995
Epoch 83/100
9/9 [==============================] - 2s 270ms/step - loss: 2.1270e-04 - accuracy: 0.9993
Epoch 84/100
9/9 [==============================] - 2s 258ms/step - loss: 1.9974e-04 - accuracy: 0.9992
Epoch 85/100
9/9 [==============================] - 2s 238ms/step - loss: 1.7557e-04 - accuracy: 0.9993
Epoch 86/100
9/9 [==============================] - 2s 244ms/step - loss: 1.9148e-04 - accuracy: 0.9992
Epoch 87/100
9/9 [==============================] - 2s 235ms/step - loss: 1.6668e-04 - accuracy: 0.9996
Epoch 88/100
9/9 [==============================] - 2s 229ms/step - loss: 1.9427e-04 - accuracy: 0.9993
Epoch 89/100
9/9 [==============================] - 2s 233ms/step - loss: 1.8

In [10]:
def to_round_number(y_hat):
    for sent in y_hat:
        for word in sent:
            for i in range(0,4):
                if word[i]>=0.5:
                    word[i]=1
                else:
                    word[i]=0
    return(y_hat)

In [11]:
def y_reconstruct(y_hat):
    y_hat = to_round_number(y_hat)
    sent_borders = []
    phrase_border = []
    for sent in y_hat:
        for word in sent:
            for i in range(0,4):
                if word[i] == 1:
                    phrase_border.append(i)
        sent_borders.append(phrase_border)
        phrase_border = []
    return(sent_borders)


In [12]:
def y_comparison(arr_to_convert,dims):
    array_converted = np.zeros((dims[0],dims[1],1))
    if not isinstance(arr_to_convert, list):
        for i in range(len(arr_to_convert)):
            for j in range(len(arr_to_convert[0])):
                for k in range(0,4):
                    if arr_to_convert[i][j][k] == 1:
                        array_converted[i][j][0] = k
    else:
        for i in range(len(arr_to_convert)):
            for j in range(len(arr_to_convert[i])):
                 array_converted[i][j][0] = arr_to_convert[i][j]
                    
    return(array_converted)



In [13]:
def phrase_building(sent):
    phrase = []
    for item in sent:
        if item[1]== 1:
            phrase.append(item[0])
        elif item[1] == 2:           
            print(' '.join(phrase))
            phrase = []
            phrase.append(item[0])
        else:
            phrase.append(item[0])
            print(' '.join(phrase))
            phrase = []
    return

In [14]:
def phrase_control(num):
    print('#############Predicted########')
    phrase_pred = list(zip(X_test[num],pred_y[num]))
    phrase_building(phrase_pred)
    print('#############Control########')
    phrase_cont = list(zip(X_test[num],test_set[num]))
    phrase_building(phrase_cont)
    return

In [15]:
X_test_pr = word_to_vector(X_test)
y_hat = model.predict(X_test_pr, verbose=0)
test_to_numb = y_to_number(y_test)
sent_borders = y_reconstruct(y_hat)
test_set = y_comparison(test_to_numb,test_to_numb.shape)
pred_y = y_comparison(sent_borders,test_to_numb.shape)
pred_y = pred_y.reshape(pred_y.shape[0],pred_y.shape[1])
test_set = test_set.reshape(test_set.shape[0],test_set.shape[1])

(134, 136, 200)
(134, 63, 4)


In [67]:
print(sent_borders[3])
print(test_set[3])
print(pred_y[3])


[1, 1, 2, 1, 2, 1, 1, 2, 1, 1, 2, 1, 2, 1, 2, 1, 1, 2, 1, 2, 1, 1, 2, 2, 1, 1, 1, 2, 1, 2, 1, 1, 1, 3]
[1. 1. 2. 1. 2. 1. 1. 2. 1. 1. 2. 1. 2. 1. 2. 1. 1. 2. 1. 2. 1. 1. 2. 2.
 1. 1. 1. 2. 1. 2. 1. 1. 1. 3. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 1. 2. 1. 2. 1. 1. 2. 2. 1. 2. 1. 2. 1. 2. 1. 2. 2. 1. 2. 2. 1. 1. 2.
 1. 2. 1. 2. 1. 2. 1. 1. 1. 3. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [60]:
nonmatch_rank = []
for i in range(len(test_set)):
    if np.any([test_set[i], pred_y[i]]) == False:
        nonmatch_rank.append(i)

print(nonmatch_rank)

[]


In [38]:
sel_idx = np.random.randint(low = 0, high = len(pred_y)+1, size = 5)
print(sel_idx)
for idx in range(0,5): 
    phrase_control(sel_idx[idx])

[ 68 101  38  39 101]
#############Predicted########
Participants
were provided
with verbal encouragement
and visual feedback
to maximize voluntary effort .
#############Control########
Participants
were provided
with verbal encouragement
and visual feedback
to maximize voluntary effort .
#############Predicted########

We
enrolled
persons
aged 40 years
or older
who met
American College of Rheumatology criteria
for symptomatic knee osteoarthritis
and had radiographic evidence
of tibiofemoral or patellofemoral osteoarthritis
( defined as the presence
of a definite osteophyte
in the tibiofemoral compartment
and/or the patellofemoral compartment ,
as assessed
on standing anterior вЂ“ posterior and lateral or sunrise views )
.
#############Control########

We
enrolled
persons
aged 40 years or older
who met
American College of Rheumatology criteria
for symptomatic knee osteoarthritis
and had radiographic evidence
of tibiofemoral or patellofemoral osteoarthritis
( defined as the presence of 

In [39]:
phrase_control(68)

#############Predicted########
Participants
were provided
with verbal encouragement
and visual feedback
to maximize voluntary effort .
#############Control########
Participants
were provided
with verbal encouragement
and visual feedback
to maximize voluntary effort .
